In [ ]:
from pathlib import Path
import pandas
import numpy
from trustutils import run

run.TRUST_parameters()

# Authorship

- Firstly version by Moncef EL MOATAMID in 2023 during his intership under the monitoring of Corentin REISS and Alan BURLOT.
- Final version by Alan BURLOT in 2024.

# Description

The objective is to test the Sato model added to a $k-\omega$ turbulence model to take bubble-induced turbulence into account. The test is made on an upward bubbly flow from the Liu \& Bankoff database. A single case is tested in this validation form, but the entire database can be tested if needed. 

The database comes from the original paper by Liu \& Bankoff (1993) : Liu, T. J. 1993 Bubble size and entrance length effects on void development in a vertical channel. Int. J. Multiphase Flow 19, 99-113 (https://doi.org/10.1016/0301-9322(93)90026-Q)

The test case is a tube with a radius 1.9 cm. The length is 1.52 meter. The fluids are liquid water for the carrying phase and air for the dispersed phase. The bubble diameters are between 2 and 4 mm. 42 flow conditions are explored with a local void fraction up to 50 \%. The water flux is $J_l \in [0.376,\ 1.391]$ and the air flux is $J_g \in [0.027,\ 0.347]$. Measured quantities are the mean liquid and gas velocities, fluctuations in axial and transverse directions with the cross-correlation. The void fraction and the Sauter diameter are also available.

To simplify the modelling process, the diameters are fixed by an interpolation procedure. This allows to avoid dealing with a interfacial area transport equation.

The validation form test in a first block the main quantities of the two-phase flow : void fraction, liquid and gas velocities, kinetic energy and turbulence cross-correlation. For the experimental data, the turbulent kinetic energy is estimated using the following expression:
$$k_{exp} = \frac{1}{2}\left(\sqrt{u_l^{'2}}^2 + 2\times \sqrt{v_l^{'2}}^2\right)$$
In a second bloc, the turbulent quantities are presented. Then a final bloc shows results on the different interfacial forces.

The results show that the extra source terms introduced in the $k$ and $\omega$ equations are better than nothing, but it does not allow capturing the turbulent kinetic in the centre of the flow as it is still dependent on the presence of a mean velocity gradient.

# User input

The computation time is limited to `tmax = 1` to obtain a result in a reasonable time.

In [ ]:
# Can be changed
force_computation = True
number_of_partitions = 6
save_figures = True
turbulence_choice = ["komega-hzdr"]  # must be a list even with one model
#subset = list(range(1, 43, 1))  # between 1 and 42
subset = [5]  # must be a list even with one case

dparam = {}
dparam["tmax"] = 1
dparam["nb_pas_dt_max"] = 1000000000
dparam["seuil_statio"] = 1e-4
dparam["solver"] = "ice"
dparam["n_rafinment"] = 2
dparam["facsec"] = 1
dparam["max_facsec"] = 1
refinement_levels = [1, 2] # two mesh levels

# Shouldn't be changed
dataroot = "jdd"
rho_l = 998.30
rho_g = 1.2
grav = -9.81

# Computation setup

## Creation of repositories

In [ ]:
if force_computation:
    run.reset()
    run.initBuildDirectory()
build = run.BUILD_DIRECTORY
%ls build

## Mesh creation

In [ ]:
run.useMEDCoupling()

In [ ]:
%%capture
if force_computation:
    %cd build/meshes
    %run canal_Liu_1.py
    %run canal_Liu_2.py 
    %ls *.med
    %cd ../../

## Definition of the trial matrix

In [ ]:
trial_matrix = pandas.read_csv("build/expdata/LiuBankoff.csv")
trial_matrix.index += 1  # to make it compliant with the Set number

# Liquid void fraction
trial_matrix["alpha_l0"] = trial_matrix['JF']/(trial_matrix['JF']+trial_matrix['JG'])

# Gas void fraction
trial_matrix["alpha_g0"] = trial_matrix['JG']/(trial_matrix['JF']+trial_matrix['JG'])

# Liquid velocity
trial_matrix["u_0"] = trial_matrix['JF']/trial_matrix["alpha_l0"]

# Boundary condition for turbulent kinetic energy
trial_matrix["CL_k"] = 0.01*trial_matrix["u_0"]**2

# Boundary condition for omega
trial_matrix["CL_om"] = trial_matrix["u_0"]/trial_matrix["D_h"]

# gravity
trial_matrix["gravity"] = grav

# Fluid couple
trial_matrix["fluid"] = "water-air"

# Name of meshes
trial_matrix["mesh"] = "Liu"

# Height of the test set
trial_matrix["height"] = 0.019*2*40

Available keys in `trial_matrix`

In [ ]:
trial_matrix.keys()

## Bubble diameters interpolation function

In [ ]:
# Function to create a predefined field of bubble diameters
nfitd = 4


def str_r_loc_d_loc(rl, dl):
    r_l_l = []
    d_l_l = []
    for ii, val in enumerate(rl):
        if (numpy.isnan(dl[ii]) == False):
            r_l_l += [val]
            d_l_l += [dl[ii]]
            
    tab_rmin_max = [0, 0]
    tab_dmin_max = [0, 0]
            
    tab_polyfitd = numpy.polyfit(r_l_l, d_l_l, nfitd)
    tab_rmin_max[0] = (r_l_l[0] + r_l_l[1] )/2.
    tab_rmin_max[1] = (r_l_l[-1] + r_l_l[-2])/2.
    tab_dmin_max[0] = (d_l_l[0] + d_l_l[1] )/2.
    tab_dmin_max[1] = (d_l_l[-1] + d_l_l[-2])/2.

    str_diam = "0."

    # First we take care of what happens above and under the highest values
    str_diam += f"+((X*X+Y*Y)<({tab_rmin_max[0]}*{tab_rmin_max[0]}))"
    str_diam += f"*{tab_dmin_max[0]}"
    str_diam += f"+((X*X+Y*Y)]({tab_rmin_max[1]}*{tab_rmin_max[1]}))"
    str_diam += f"*{tab_dmin_max[1]}"

    # Then we take care of the middle
    str_loc = "0."
    for ii in range(nfitd + 1):
        str_loc += f"+({tab_polyfitd[ii]})"
        for jj in range(nfitd - ii):
            str_loc+= "*sqrt(x*x+y*y)"
            
    str_diam += f"+((X*X+Y*Y)]({tab_rmin_max[0]}*{tab_rmin_max[0]}))" # put to zero outside the right interval
    str_diam += f"*((X*X+Y*Y)<({tab_rmin_max[1]}*{tab_rmin_max[1]}))" 
    str_diam += f"*({str_loc})"          
    str_diam = f"1.e-3*({str_diam})" # mm => m
    return str_diam

# Bubble diameter
tab_str_diam = ["1.e-3"]*len(trial_matrix["SetNumber"]) # Default single phase
for index, row in trial_matrix.iterrows():
    expNumber = row["SetNumber"]
    path = f"build/expdata/Set{expNumber[1:]}/Exp.csv"
    tab_G = pandas.read_csv(path, sep = ',')

    r_loc = numpy.array(tab_G["r/R"])*row['D_h']/2.
    d_loc = numpy.array(tab_G["Dav"]) # These are in mm

    tab_str_diam[index-1] = str_r_loc_d_loc(r_loc, d_loc)

trial_matrix["Dbubble"] = tab_str_diam

## Definition of substitution dictionaries

In [ ]:
# Modèles
def dico_model():
    dico = {}
    dico["interface"] = "interface_eau_air interface_sigma_constant { tension_superficielle 0.0728 }"
    dico["carrying_phase"] = "liquide_eau Fluide_Incompressible { mu champ_uniforme 1 1.002e-3 rho champ_uniforme 1 998.30 lambda Champ_Uniforme 1 0.604 Cp Champ_Uniforme 1 75.366 beta_th Champ_Uniforme 1 0 }"
    dico["dispersed_phase"] = "gaz_air Fluide_Incompressible { mu champ_uniforme 1 1.85e-5 rho champ_uniforme 1 1.2 lambda Champ_Uniforme 1 0.023 Cp Champ_Uniforme 1 1006 beta_th Champ_Uniforme 1 0 }"
    dico["frottement_interfacial"] = "Tomiyama { contamination 2 }"
    dico["masse_ajoutee"] = "coef_constant { }"
    dico["portance_interfaciale"] = "Tomiyama { }"
    dico["dispersion_bulles"] = "turbulente_burns { }"
    dico["WLu"] = 'paroi_frottante_loi { }'
    dico["beta_portance"] = 1
    dico["beta_disp"] = 1
    dico["beta_wall_disp"] = 1
    dico["beta_wall_lift"] = 1
    return dico
    
def dico_trial_matrix(row):
    dico = {}
    dico["diametre_bulles"] = row["Dbubble"]
    dico["u_0"] = row["u_0"]
    dico["grav"] = row["gravity"]
    dico["alpha_l0"] = row["alpha_l0"]
    dico["alpha_v0"] = row["alpha_g0"]
    dico["CI_k"] = row["CL_k"]
    dico["CI_diss"] = row["CL_om"]
    dico["h_sonde"] = str(row["height"]*0.947986)
    dico["x_sonde"] = str(row["D_h"]/2*numpy.cos(2.5*numpy.pi/180))
    dico["y_sonde"] = str(row["D_h"]/2*numpy.sin(2.5*numpy.pi/180))
    return dico

def dico_turbulence_model(model_name, row):
    dico = {}
    dico["CeHZDR"] = 1.0
    dico["CkHZDR"] = 0.002
    return dico

# Computation launcher

In [ ]:
if force_computation:
    dmodel = dico_model()
    for index, row in trial_matrix.iterrows(): # all experiment case
        if index in subset:
            dico_tm = dico_trial_matrix(row)
            for turbmod in turbulence_choice: # all turbulence models
                dturb = dico_turbulence_model(turbmod, row)
                for ref in refinement_levels: # all refinements level
                    casename = f"LB_S{row['SetNumber']}_{turbmod}/m{ref}"
                    dico = {}
                    dico["name_mesh"] = f"../../meshes/mesh_Liu_{ref}.med"
    
    
                    dd = {**dparam, **dmodel, **dico_tm, **dturb, **dico}
    
                    myrun = run.addCaseFromTemplate(f"{dataroot}-{turbmod}.data",
                                                    casename,
                                                    dd,
                                                    nbProcs=number_of_partitions)
                    if number_of_partitions > 1:
                        myrun.partition()
    run.printCases()

In [ ]:
if force_computation:
    run.runCases()

## Computation statistics

In [ ]:
if force_computation:
    table = run.tablePerf()
    table.index = [aaa.split("/")[0] for aaa in table.index]
    table

## Residuals

In [ ]:
from trustutils import visit
from trustutils.jupyter import plot
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import os
print(os.getcwd())

In [ ]:
plt.close()
fig = plt.figure(figsize = (16, 4))
color_raf = ["red", "green", "blue"]
name_phys = [["vitl", "vitg"], 
             ["alphal", "alphag"],  
             ["k"], 
             ["diss"]]
par = "PAR_"
if number_of_partitions == 1 : 
    par = ""

unfoundedfiles = []

axs = fig.subplots(1, 4, sharex=True, sharey=True)

for ii, nn in enumerate(name_phys):
    for jj, row in trial_matrix.iterrows():
        if jj in subset: # only the selected cases
            for turbmod in turbulence_choice:
                for rr in refinement_levels:
                    repo = f"LB_S{row['SetNumber']}_{turbmod}/m{rr}"
                    ficname = f"build/{repo}/{par}{dataroot}-{turbmod}.dt_ev"
                    try:
                        residuals = pandas.read_csv(f"build/{repo}/{par}{dataroot}-{turbmod}.dt_ev", sep="\t")
                        residuals = residuals.iloc[:,:13]
                        residuals.columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg', 'alphal', 'alphag', 'Tl', 'Tg', 'diss', 'k']               
                        label_loc = repo
                        for ll, vv in enumerate(nn) : #physique de la phase si multiphase
                            axs[ii].plot(residuals["time"][::], residuals[vv][::], "-", 
                                         label=label_loc, color=color_raf[rr])
                    except FileNotFoundError as e:
                        print(e)
                        unfoundedfiles.append(ficname)

    axs[ii].set_xlim(0, max(residuals["time"]))
    #axs[ii].set_ylim(float(dparam["seuil_statio"])/100, 1e4)
    axs[ii].set_yscale("log")
    axs[ii].set_title(f"convergence {nn[0]}")
    axs[ii].set_xlabel('time')
    axs[ii].set_ylabel(f"convergence {nn[0]}")
    #axs[l].legend(prop={'size': 10})

plt.tight_layout()
if save_figures:
    plt.savefig(f"build/convergence.pdf")
plt.show()


The available experimental data are

In [ ]:
exp, expTurb = {}, {}
for n in subset: # experience
    exp[n] = pandas.read_csv(f"{build}/expdata/Set{n}/Exp.csv",
                             sep=",")
    expTurb[n] = pandas.read_csv(f"{build}/expdata/Set{n}/ExpTurb.csv",
                                 sep=",")
exp

In [ ]:
# list of gas and liquid flow rates
listjg = numpy.unique([trial_matrix["JG"][num] for _, num in enumerate(subset)])
listjl = numpy.unique([trial_matrix["JF"][num] for _, num in enumerate(subset)])


def get_idx_jljg(num, jz):
    if jz.lower() not in ["jf", "jl", "jg"]:
        raise ValueError("Allowed values are jl, jf and jg.")
    if jz.upper() in ["JF", "JL"]:
        thelist = listjl
        choice = "JF"
    else:
        thelist = listjg
        choice = "JG"

    return numpy.abs(thelist - trial_matrix[choice][num]).argmin()

# get_idx_jljg(40, "jl")

In [ ]:
trial_matrix.index

In [ ]:
import numpy as np
from pathlib import Path
simres = {}
sonde_gradv = sonde_k_WIT = sonde_epsilon_WIT = sonde_F = sonde_BIA = True

for jj, row in trial_matrix.iterrows():
    if jj in subset: # only the selected cases
        simres[jj] = {}
        for turbmod in turbulence_choice:
            simres[jj][turbmod] = {}
            for rr in refinement_levels:
                repo = f"build/LB_S{row['SetNumber']}_{turbmod}/m{rr}"
                name = f"{dataroot}-{turbmod}"
                
                if not Path(f"{repo}/PAR_{name}_DP.son").is_file():
                    continue
                
                simres[jj][turbmod][rr] = pandas.DataFrame()
                
                simres[jj][turbmod][rr]['r+'] = np.linspace(0, 1, 100)
                simres[jj][turbmod][rr]['dp'] = np.loadtxt(f"{repo}/PAR_{name}_DP.son")[-1][1:]
                
                # vitesses
                simres[jj][turbmod][rr]['vxl'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_EAU.son")[-1][1::3]
                simres[jj][turbmod][rr]['vyl'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_EAU.son")[-1][2::3]
                simres[jj][turbmod][rr]['vzl'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_EAU.son")[-1][3::3]
                
                simres[jj][turbmod][rr]['vxg'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_AIR.son")[-1][1::3]
                simres[jj][turbmod][rr]['vyg'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_AIR.son")[-1][2::3]
                simres[jj][turbmod][rr]['vzg'] = np.loadtxt(f"{repo}/PAR_{name}_VITESSE_AIR.son")[-1][3::3]
                
                # bulles
                simres[jj][turbmod][rr]['DB']  = 1000*np.loadtxt(f"{repo}/PAR_{name}_DIAMETRE.son")[-1][2::2]
                simres[jj][turbmod][rr]['alp'] = np.loadtxt(f"{repo}/PAR_{name}_ALPHA_AIR.son")[-1][1::]
                
                # turbulence
                simres[jj][turbmod][rr]['k_SIT'] = np.loadtxt(f"{repo}/PAR_{name}_K.son")[-1][1::]
                simres[jj][turbmod][rr]['omega_SIT'] = np.loadtxt(f"{repo}/PAR_{name}_DISS.son")[-1][1::]
                # simres[jj][turbmod][rr]['nu_t'] = np.loadtxt(f"{repo}/PAR_{name}_VISCOSITE.son")[-1][1::]
                
                # qdm
                simres[jj][turbmod][rr]['conv'] = np.loadtxt(f"{repo}/PAR_{name}_CONV_V.son")[-1][1::6]
                simres[jj][turbmod][rr]['diff'] = np.loadtxt(f"{repo}/PAR_{name}_DIFF_V.son")[-1][1::6]

                # if a probe for gradV is there
                try:
                    simres[jj][turbmod][rr]['dvxl_dx'] = np.loadtxt(f"{repo}/PAR_{name}_GRADV.son")[-1][1::9]
                    simres[jj][turbmod][rr]['dvyl_dy'] = np.loadtxt(f"{repo}/PAR_{name}_GRADV.son")[-1][4::9]
                    simres[jj][turbmod][rr]['dvzl_dz'] = np.loadtxt(f"{repo}/PAR_{name}_GRADV.son")[-1][9::9]
                    simres[jj][turbmod][rr]['dvzl_dx'] = np.loadtxt(f"{repo}/PAR_{name}_GRADV.son")[-1][7::9]
                    simres[jj][turbmod][rr]['upvp'] = simres[jj][turbmod][rr]['nu_t']*np.abs(simres[jj][turbmod][rr]['dvzl_dx']) # frottement turbulent u'v'
                except:
                    simres[jj][turbmod][rr]['dvxl_dx'] = np.zeros(100)
                    simres[jj][turbmod][rr]['dvyl_dy'] = np.zeros(100)
                    simres[jj][turbmod][rr]['dvzl_dz'] = np.zeros(100)
                    simres[jj][turbmod][rr]['dvzl_dx'] = np.zeros(100)
                    simres[jj][turbmod][rr]['upvp'] = np.zeros(100)
                    sonde_gradv = False

                # if HZDR model
                try:
                    simres[jj][turbmod][rr]['k_HZDR'] = np.loadtxt(f"{repo}/PAR_{name}_PROD_K_HZDR.son")[-1][1::]
                except:
                    simres[jj][turbmod][rr]['k_HZDR'] = np.zeros(100)
                    sonde_k_WIT = False

                # if du Cluzeau's model is used
                try:
                    simres[jj][turbmod][rr]['k_WIT'] = np.loadtxt(f"{repo}/PAR_{name}_K_WIT.son")[-1][1::]
                except:
                    simres[jj][turbmod][rr]['k_WIT'] = np.zeros(100)
                    sonde_k_WIT = False

                try:
                    simres[jj][turbmod][rr]['epsilon_WIT'] = np.loadtxt(f"{repo}/PAR_{name}_DISS_K_WIT.son")[-1][1::]
                except:
                    simres[jj][turbmod][rr]['epsilon_WIT'] = np.zeros(100)
                    sonde_epsilon_WIT = False

               # Calcul de k_tot (SIT+BIA)
                simres[jj][turbmod][rr]['ur'] = numpy.sqrt((simres[jj][turbmod][rr]['vxg'] - simres[jj][turbmod][rr]['vxl'])**2 
                                                  + (simres[jj][turbmod][rr]['vyg'] - simres[jj][turbmod][rr]['vyl'])**2 
                                                  + (simres[jj][turbmod][rr]['vzg'] - simres[jj][turbmod][rr]['vzl'])**2)
                simres[jj][turbmod][rr]['k_tot'] = simres[jj][turbmod][rr]['k_SIT']
                if "wit" in turbmod:
                    simres[jj][turbmod][rr]['k_tot'] += simres[jj][turbmod][rr]['k_WIT']
                if "wif" in turbmod:
                    simres[jj][turbmod][rr]['k_WIF'] = (9/20 + (1/20+3/2*0.25)) * simres[jj][turbmod][rr]['alp'] * simres[jj][turbmod][rr]['ur']**2
                    simres[jj][turbmod][rr]['k_tot'] += simres[jj][turbmod][rr]['k_WIF']
                else:
                    simres[jj][turbmod][rr]['k_WIF'] = np.zeros(100)
                    
                # if interfacial forces are in sondes
                try:
                    simres[jj][turbmod][rr]['drag'] = np.array(np.loadtxt(f"{repo}/PAR_{name}_DRAG.son")).T[1::6, -1] # composante selon x - phase liquide
                    simres[jj][turbmod][rr]['lift'] = np.array(np.loadtxt(f"{repo}/PAR_{name}_LIFT.son")).T[1::6, -1] # composante selon x - phase liquide
                    simres[jj][turbmod][rr]['disp'] = np.array(np.loadtxt(f"{repo}/PAR_{name}_DISP.son")).T[1::6, -1] # composante selon x - phase liquide
                    simres[jj][turbmod][rr]['lub'] = np.array(np.loadtxt(f"{repo}/PAR_{name}_LUB.son")).T[1::6, -1]   # composante selon x - phase liquide
                except:
                    simres[jj][turbmod][rr]['drag'] = np.zeros(100)
                    simres[jj][turbmod][rr]['lift'] = np.zeros(100)
                    simres[jj][turbmod][rr]['disp'] = np.zeros(100)
                    simres[jj][turbmod][rr]['lub'] = np.zeros(100)
                    sonde_F = False

                # if BIA source term is in sondes : div(-Rij_BIA)
                try:
                    simres[jj][turbmod][rr]['BIF'] = np.array(np.loadtxt(f"{repo}/PAR_{name}_BIF.son")).T[1::6, -1] # composante selon x - phase liquide
                except:
                    simres[jj][turbmod][rr]['BIF'] = np.zeros(100)
                    sonde_BIA = False


Available keys are

In [ ]:
print(simres[list(simres.keys())[0]][turbulence_choice[0]][refinement_levels[0]].keys())

The following keys are null:

In [ ]:
for jj, _ in trial_matrix.iterrows():
    if jj in subset: # only the selected cases
        for turbmod in turbulence_choice:
            for rr in refinement_levels:
                myres = simres[jj][turbmod][rr]
                for key, val in myres.items():
                    if val.eq(0).all():
                        print(f"{jj}, {turbmod}, {rr} : {key} is null")

# Comparison between experiments and simulations

In [ ]:
# Help manage cases if only a single experimental condition is computed
def get_axis(axs, ch, idx):
    if ncols == 1:
        return axs[ch]
    else:
        return axs[ch, idx]
    

ml = 2  # mesh level to be plotted

# custom plot
ms = 3  # markersize

# Sort the SetNumber to use jl and jg
label_jl = [r"Liu $J_L = 0.376$ (ms$^{-1}$)",
            r"Liu $J_L = 0.535$ (ms$^{-1}$)",
            r"Liu $J_L = 0.753$ (ms$^{-1}$)",
            r"Liu $J_L = 0.974$ (ms$^{-1}$)",
            r"Liu $J_L = 1.087$ (ms$^{-1}$)",
            r"Liu $J_L = 1.391$ (ms$^{-1}$)"]
label_jg = [r"$J_G = 0.027$ ms$^{-1}$",
            r"$J_G = 0.067$ ms$^{-1}$",
            r"$J_G = 0.112$ ms$^{-1}$",
            r"$J_G = 0.180$ ms$^{-1}$",
            r"$J_G = 0.230$ ms$^{-1}$",
            r"$J_G = 0.293$ ms$^{-1}$",
            r"$J_G = 0.347$ ms$^{-1}$"]

# axs(number of quantities to plot, number of J_L conditions)
nrows = 7
ncols = len(numpy.unique([trial_matrix["JF"][num] for _, num in enumerate(subset)]))
jgcolor = ("blue", "orange", "green", "indigo", "red", "chartreuse", "brown")
print(f"We have {ncols} liquid conditions")

In [ ]:
def plot_main_quantities_for_model(turbmod):
    fig, axs = plt.subplots(nrows, ncols, sharey="row", figsize=(28, 20))
    for ii, num in enumerate(subset):

        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
        idx = get_idx_jljg(num, "jl")
        idxjg = get_idx_jljg(num, "jg")

        myaxs = get_axis(axs, 0, idx)
        myaxs.plot([-1, -2],[-1, -1], "o-", 
                   color=jgcolor[idxjg],
                   label = f"{trial_matrix['SetNumber'][num]}: {trial_matrix[trial_matrix['SetNumber'] == f'L{num}']['JG'].values[0]} m/s",
                   markerfacecolor = "White")

        # Void fraction
        myaxs = get_axis(axs, 1, idx)
        myaxs.plot(myres["r+"], myres["alp"], color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], exp[num]['alpha'], "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.set_xlim(0, 1)
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # d_B(r/R)   : bubble diameter
        myaxs = get_axis(axs, 2, idx)
        myaxs.plot(myres['r+'], myres['DB'], color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], exp[num]['Dav'], "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # U_L(r/R)   : mean liquid velocity
        myaxs = get_axis(axs, 3, idx)
        myaxs.plot(myres['r+'], myres['vzl'], color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], exp[num]['Uf'], "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # U_G(r/R)   : mean air velocity
        myaxs = get_axis(axs, 4, idx)
        myaxs.plot(myres['r+'], myres['ur'], color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], exp[num]['Ub'] - exp[n]['Uf'], "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # Kinetic energy
        myaxs = get_axis(axs, 5, idx)
        myaxs.plot(myres['r+'], myres['k_tot'], color=jgcolor[idxjg])
        myaxs.plot(expTurb[num]['r/R'], 0.5*(expTurb[num]['uf']*expTurb[num]['uf'] + 2*expTurb[num]['vf']*expTurb[num]['vf']), 
                   "o", color=jgcolor[idxjg], markersize=ms, markerfacecolor="white")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # u'v'
        myaxs = get_axis(axs, 6, idx)
        myaxs.plot(myres['r+'], myres['upvp'], color=jgcolor[idxjg])
        myaxs.plot(expTurb[num]['r/R'], expTurb[num]['uv'], "o", markersize = ms, markerfacecolor = "black")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row


    # Customization
    for ii, num in enumerate(subset):
        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
        idx = get_idx_jljg(num, "jl")
        myaxs = get_axis(axs, 0, idx)
        myaxs.text(0,0.1, label_jl[idx], ha = "center", va = "center", fontsize = 14)
        myaxs.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.1), fontsize = 12, ncol=1)
        myaxs.set_xlim(-.5,.5)
        myaxs.set_ylim(-.7,.3)
        myaxs.axis('off')

        myaxs = get_axis(axs, 6, idx)
        myaxs.set_xlabel(r"$r/R$")

    ylab_fs = 18
    ylab_pad = 15
    myaxs = get_axis(axs, 1, 0)
    myaxs.set_ylabel(r"$\alpha$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 2, 0)
    myaxs.set_ylabel(r"$D_S$ (mm)", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 3, 0)
    myaxs.set_ylabel(r"$u_l$ (m/s)", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 4, 0)
    myaxs.set_ylabel(r"$u_g$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 5, 0)
    myaxs.set_ylabel(r"$k$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 6, 0)
    myaxs.set_ylabel(r"$u'v'$", fontsize=ylab_fs, labelpad=ylab_pad)

    plt.tight_layout()

In [ ]:
plot_main_quantities_for_model("komega-hzdr")

# Turbulence quantities

We plot quantities for each turbulence model and compare some of them between models.

Determine the number of different liquid flow rate to be plotted.

In [ ]:
lst_computed_jl = np.unique([get_idx_jljg(num, "jl") for num in subset])
lst_computed_jg = np.unique([get_idx_jljg(num, "jg") for num in subset])
n_jl = len(lst_computed_jl)
n_jg = len(lst_computed_jg)

def get_index_in_list(thelist, val):
    return np.abs(thelist - val).argmin()

In [ ]:
def plot_turbulence_quantities_for_model(turbmod):
        
    nrows = 4  # legend, k_SIT/ur^2, k_SIT/alpha/ur^2, 0.9*omega_SIT*k_SIT
    figheight = 14
    
    nrowssupp = 1  # because HZDR model is used   
        
    nrows += nrowssupp
    figheight += nrowssupp*3
        
    fig, axs = plt.subplots(nrows, ncols, sharey="row", figsize=(28, figheight))
    for ii, num in enumerate(subset):

        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
            
        idx = get_idx_jljg(num, "jl")
        idxjg = get_idx_jljg(num, "jg")
        k_exp = 0.5*(expTurb[num]["uf"]**2 + 2*expTurb[num]["vf"]**2)
        ur_exp = exp[num]["Ub"] - exp[num]["Uf"]

        # 0 :  
        myaxs = get_axis(axs, 0, idx)
        myaxs.plot([-1, -2],[-1, -1], "o-", 
                   color=jgcolor[idxjg],
                   label = f"{trial_matrix['SetNumber'][num]}: {trial_matrix[trial_matrix['SetNumber'] == f'L{num}']['JG'].values[0]} m/s",
                   markerfacecolor = "White")

        # k_SIT/u_r^2
        myaxs = get_axis(axs, 1, idx)
        myaxs.plot(myres["r+"], myres["k_SIT"]/myres["ur"]**2, color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], k_exp/ur_exp**2, "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.set_xlim(0, 1)
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # k_SIT/(alpha*u_r^2)
        myaxs = get_axis(axs, 2, idx)
        myaxs.plot(myres['r+'], myres["k_SIT"]/(myres["alp"]*myres["ur"]**2), color=jgcolor[idxjg])
        myaxs.plot(exp[num]['r/R'], k_exp/(exp[num]["alpha"]*ur_exp**2), "o", color=jgcolor[idxjg], 
                   markersize=ms, markerfacecolor="white")
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks
        
        # 0.9*omega_SIT*k_SIT
        myaxs = get_axis(axs, 3, idx)
        myaxs.plot(myres['r+'], 0.9*myres["omega_SIT"]*myres["k_SIT"], color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)
        
        myaxs = get_axis(axs, 4, idx)
        myaxs.plot(myres['r+'], myres["k_SIT"], linestyle="solid", color=jgcolor[idxjg])
        myaxs.plot(myres['r+'], myres["k_HZDR"], linestyle="dashdot", color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        
    # Customization
    for ii, num in enumerate(subset):
        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue

        idx = get_idx_jljg(num, "jl")
        myaxs = get_axis(axs, 0, idx)
        myaxs.text(0, 0.1, label_jl[idx], ha="center", va="center", fontsize=14)
        myaxs.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.1), fontsize = 12, ncol=1)
        myaxs.set_xlim(-.5,.5)
        myaxs.set_ylim(-.7,.3)
        myaxs.axis('off')

        myaxs = get_axis(axs, nrows-1, idx)
        myaxs.set_xlabel(r"$r/R$")

    ylab_fs = 18
    ylab_pad = 15
    myaxs = get_axis(axs, 1, 0)
    myaxs.set_ylabel(r"$k_{SIT}/u_r^2$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 2, 0)
    myaxs.set_ylabel(r"$k_{SIT}/(\alpha u_r^2)$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 3, 0)
    myaxs.set_ylabel(r"$0.9*\omega_{SIT}*k_{SIT}$", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 4, 0)
    myaxs.set_ylabel(r"$k_{SIT}$ vs $k_{HZDR}$", fontsize=ylab_fs, labelpad=ylab_pad)
        
    #fig.subplots_adjust(top=0.9)
    plt.tight_layout()
    

In [ ]:
plot_turbulence_quantities_for_model("komega-hzdr")

# Interfacial forces vs turbulence

In [ ]:
def plot_interfacial_forces_for_model(turbmod):

    nrowssupp = 1
        
    nrows = 5 + nrowssupp  # legend, k_SIT/ur^2, k_SIT/alpha/ur^2, 0.9*omega_SIT*k_SIT
    figheight = 15 + nrowssupp*3

    fig, axs = plt.subplots(nrows, ncols, sharey="row", figsize=(28, figheight))
    for ii, num in enumerate(subset):

        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
        idx = get_idx_jljg(num, "jl")
        idxjg = get_idx_jljg(num, "jg")
        k_exp = 0.5*(expTurb[num]["uf"]**2 + 2*expTurb[num]["vf"]**2)
        ur_exp = exp[num]["Ub"] - exp[num]["Uf"]

        # 0 :  
        myaxs = get_axis(axs, 0, idx)
        myaxs.plot([-1, -2],[-1, -1], "o-", 
                   color=jgcolor[idxjg],
                   label = f"{trial_matrix['SetNumber'][num]}: {trial_matrix[trial_matrix['SetNumber'] == f'L{num}']['JG'].values[0]} m/s",
                   markerfacecolor = "White")

        # drag
        myaxs = get_axis(axs, 1, idx)
        myaxs.plot(myres["r+"], myres["drag"], color=jgcolor[idxjg])
        myaxs.set_xlim(0, 1)
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # lift
        myaxs = get_axis(axs, 2, idx)
        myaxs.plot(myres['r+'], myres["lift"], color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # disp
        myaxs = get_axis(axs, 3, idx)
        myaxs.plot(myres['r+'], myres["disp"], color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks

        # disp
        myaxs = get_axis(axs, 4, idx)
        myaxs.plot(myres['r+'], myres["lub"], color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks
            
        # BIA vs forces
        myaxs = get_axis(axs, 5, idx)
        myaxs.plot(myres['r+'], myres["drag"] + myres["lift"] + myres["disp"] + myres["lub"], color=jgcolor[idxjg])
        myaxs.plot(myres['r+'], myres["BIF"], linestyle='dashed', color=jgcolor[idxjg])
        myaxs.sharex(get_axis(axs, 1, idx))  # to share the x-axis, but not with the first row
        
    # Customization
    for ii, num in enumerate(subset):
        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
        idx = get_idx_jljg(num, "jl")
        myaxs = get_axis(axs, 0, idx)
        myaxs.text(0, 0.1, label_jl[idx], ha="center", va="center", fontsize=14)
        myaxs.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.1), fontsize = 12, ncol=1)
        myaxs.set_xlim(-.5,.5)
        myaxs.set_ylim(-.7,.3)
        myaxs.axis('off')

        myaxs = get_axis(axs, nrows-1, idx)
        myaxs.set_xlabel(r"$r/R$")

    ylab_fs = 18
    ylab_pad = 15
    myaxs = get_axis(axs, 1, 0)
    myaxs.set_ylabel(r"Drag", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 2, 0)
    myaxs.set_ylabel(r"Lift", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 3, 0)
    myaxs.set_ylabel(r"Disp", fontsize=ylab_fs, labelpad=ylab_pad)
    myaxs = get_axis(axs, 4, 0)
    myaxs.set_ylabel(r"Lub", fontsize=ylab_fs, labelpad=ylab_pad)    
        
    #fig.subplots_adjust(top=0.9)
    plt.tight_layout()

In [ ]:
plot_interfacial_forces_for_model("komega-hzdr")

# Sum of momentum equation

In [ ]:
def plot_momentum_balance(turbmod):
        
    nrows = 2  # legend, k_SIT/ur^2, k_SIT/alpha/ur^2, 0.9*omega_SIT*k_SIT
    figheight = 2 + nrows*3

    fig, axs = plt.subplots(nrows, ncols, sharey="row", figsize=(28, figheight))
    for ii, num in enumerate(subset):
        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
            
        idx = get_idx_jljg(num, "jl")
        idxjg = get_idx_jljg(num, "jg")

        # 0 :  
        myaxs = get_axis(axs, 0, idx)
        myaxs.plot([-1, -2],[-1, -1], "o-", 
                   color=jgcolor[idxjg],
                   label = f"{trial_matrix['SetNumber'][num]}: {trial_matrix[trial_matrix['SetNumber'] == f'L{num}']['JG'].values[0]} m/s",
                   markerfacecolor = "White")

        # drag
        myaxs = get_axis(axs, 1, idx)
        mombal = myres["drag"] + myres["lift"] + myres["disp"] + myres["lub"] + myres["BIF"] + myres["conv"] + myres["diff"]
        myaxs.plot(myres["r+"], mombal, color=jgcolor[idxjg])
        myaxs.set_xlim(0, 1)
        plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks
        #plt.setp(myaxs.get_xticklabels(), visible=False)  # to avoid showing the ticks
        
    # Customization
    for ii, num in enumerate(subset):
        try:
            myres = simres[num][turbmod][ml]
        except KeyError as e:
            continue
        idx = get_idx_jljg(num, "jl")
        myaxs = get_axis(axs, 0, idx)
        myaxs.text(0, 0.1, label_jl[idx], ha="center", va="center", fontsize=14)
        myaxs.legend(loc = 'lower center', bbox_to_anchor=(0.5, -0.1), fontsize = 12, ncol=1)
        myaxs.set_xlim(-.5,.5)
        myaxs.set_ylim(-.7,.3)
        myaxs.axis('off')

        myaxs = get_axis(axs, nrows-1, idx)
        myaxs.set_xlabel(r"$r/R$")

    ylab_fs = 18
    ylab_pad = 15
    myaxs = get_axis(axs, 1, 0)
    myaxs.set_ylabel(r"Momentum balance", fontsize=ylab_fs, labelpad=ylab_pad)
        
    #fig.subplots_adjust(top=0.9)
    plt.tight_layout()

In [ ]:
plot_momentum_balance("komega-hzdr")